In [187]:
import numpy as np
import pandas as pd
import json
import time

### How do we predict what rating a user is going to give?
1) Business side:
    - location
    - type
    - hours
    - parking availability
    - ambience
    - takeout?
2) User side:
    - how long on yelp
    - elite status
    - review count
    - useful/funny/cool
    - home

# TODO
    - Matthew
        - isolate restaurants in business_df
        - preprocessing: marking reviews as good or bad
    - feature extraction
        - hometown
        - number of tips by user

    - other preprocessing
        - yelpin since: timestamp today - yelping since --- COMPLETE
        - elite status: how many years have they been elite
        - only consider business with certain number of reviews
        - only consider "active users"

# Next
    - what happens if we remove "inactive" users

        

In [165]:
business_df = pd.read_csv("sample_restaurant.csv")
checkin_df = pd.read_csv("sample_checkin.csv")
review_df = pd.read_csv("sample_review.csv")
tip_df = pd.read_csv("sample_tip.csv")
user_df = pd.read_csv("sample_user.csv")

In [166]:
business_df

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,126141,9XotjNYdPzWCokrVMRo4Mg,The Great Canadian Bagel,"120 Wye Road, Unit 101",Sherwood Park,AB,T8A 6P2,53.512479,-113.327738,4.0,9,1,"{'BikeParking': 'True', 'OutdoorSeating': 'Fal...","Breakfast & Brunch, Sandwiches, Bagels, Coffee...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
1,8518,Y92Nc9GhuXWvIABqNtSWGA,Tavern 52,329 Kennett Pike,Mendenhall,PA,19357,39.853403,-75.640789,3.5,21,1,"{'RestaurantsReservations': 'True', 'WiFi': ""u...","Breakfast & Brunch, Restaurants, American (Tra...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
2,11669,b5byt9ewnM1U57ygWTTV2g,Crab Du Jour - Delran,4000 US-130,Delran,NJ,08075,40.011577,-74.961777,3.0,25,1,"{'Alcohol': ""u'full_bar'"", 'Caters': 'True', '...","Live/Raw Food, Cajun/Creole, Seafood, Restaurants","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
3,116916,4jICs7no7322VjY4hqHVAg,La Cocinita Cubana,113 Central Dr,Brandon,FL,33510,27.939235,-82.292208,4.5,33,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Spanish, Cuban, Restaurants","{'Monday': '7:0-15:30', 'Tuesday': '7:0-15:30'..."
4,66207,pWDE_rueENu_f5HKA_AsWw,Dune Coffee Roasters - Storke,250 A Storke Rd,Goleta,CA,93117,34.429540,-119.869271,4.0,159,1,"{'NoiseLevel': ""u'average'"", 'Alcohol': ""'none...","Food, Restaurants, Cafes, Coffee Roasteries, C...","{'Monday': '6:30-16:0', 'Tuesday': '6:30-16:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,141107,guwb1zuNEMtJVproo0ntFw,La Boca,870 Tchoupitoulas St,New Orleans,LA,70130,29.943455,-90.067341,4.0,420,1,"{'RestaurantsReservations': 'True', 'WiFi': ""u...","Argentine, Latin American, Steakhouses, Desser...","{'Tuesday': '17:30-22:0', 'Wednesday': '17:30-..."
39996,66695,-78Va3llKcPbL6SW4is8lw,Mungo's Italian Eatery,100 East Main St,Collinsville,IL,62234,38.669906,-89.985827,4.0,124,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Seafood, Steakhouses, Italian, Salad, Restaurants","{'Monday': '16:0-21:0', 'Tuesday': '16:0-21:0'..."
39997,101198,2pj5byDMoMDu-d0bTu2Sew,Pip,10403 83 Ave NW,Edmonton,AB,T6E 2C7,53.518958,-113.497891,4.0,117,1,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Comfort Food, Food, Wine Bars, Breakfast & Bru...","{'Monday': '9:0-23:0', 'Tuesday': '9:0-23:0', ..."
39998,124342,RKRy17pwMJAbehSkZglwjw,Jia's Wok,477 E Plumb Ln,Reno,NV,89502,39.506363,-119.797213,3.0,35,1,"{'Alcohol': ""u'none'"", 'RestaurantsGoodForGrou...","Chinese, Restaurants","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."


In [167]:
business_df.drop(columns = 'Unnamed: 0', inplace = True)
business_loc = business_df.iloc[:, [0,3, 4, 5, 6, 7]]

In [168]:
user_useful = user_df.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
user_useful.set_index('user_id', inplace = True)

In [170]:
review_df.drop(columns = 'Unnamed: 0', inplace=True)

In [171]:
rJoinU = review_df.join(user_useful, on = 'user_id', lsuffix='_Rev', rsuffix = '_User')

In [172]:
rJoinU.dropna(inplace = True)

In [173]:
business_loc.set_index('business_id', inplace= True)

In [174]:
rJUJBloc = rJoinU.join(business_loc, on='business_id', lsuffix='_rJU', rsuffix='_bloc')

In [175]:
rJUJBloc.dropna(inplace= True)

In [176]:
rJUJBloc

,review_id,user_id,business_id,stars,useful_Rev,funny_Rev,cool_Rev,text,date,name,...,cool_User,elite,friends,fans,average_stars,city,state,postal_code,latitude,longitude
8,NILjwITiFhpdhiE3SiHR8Q,WUgxsRUFjATha8L4qWYDww,fbQkVymvZ0dig8umltKhZQ,3,2,1,1,They give you LOTS of food and you will not go...,2020-09-23 21:49:15,Glenn,...,552,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...","dXD3TU8-FR4NnIzlkRk_3g, nSRPpreSnTaAMrEx0YvdnA...",31,3.57,Tampa,FL,33609,27.948237,-82.527587
19,hOwUw1fz3aB4ZhgqCetGTw,fr1Hz2acAb3OaL3l6DyKNg,RewAum_fQ0lXdFoUX18J0g,5,17,4,13,It is 4.5 stars. Not yet 5 stars. Mel has a go...,2020-08-18 19:09:46,Boon,...,27824,"2014,2015,2016,2017,2018,2019,20,20,2021","tuoDDqTDCB-lDgelvMrp5Q, 0yyXKmdI9lHxFKzomy6yfw...",387,4.04,Tampa,FL,33610,28.031316,-82.451534
19,hOwUw1fz3aB4ZhgqCetGTw,fr1Hz2acAb3OaL3l6DyKNg,RewAum_fQ0lXdFoUX18J0g,5,17,4,13,It is 4.5 stars. Not yet 5 stars. Mel has a go...,2020-08-18 19:09:46,Boon,...,27824,"2014,2015,2016,2017,2018,2019,20,20,2021","tuoDDqTDCB-lDgelvMrp5Q, 0yyXKmdI9lHxFKzomy6yfw...",387,4.04,Tampa,FL,33610,28.031316,-82.451534
19,hOwUw1fz3aB4ZhgqCetGTw,fr1Hz2acAb3OaL3l6DyKNg,RewAum_fQ0lXdFoUX18J0g,5,17,4,13,It is 4.5 stars. Not yet 5 stars. Mel has a go...,2020-08-18 19:09:46,Boon,...,27824,"2014,2015,2016,2017,2018,2019,20,20,2021","tuoDDqTDCB-lDgelvMrp5Q, 0yyXKmdI9lHxFKzomy6yfw...",387,4.04,Tampa,FL,33610,28.031316,-82.451534
19,hOwUw1fz3aB4ZhgqCetGTw,fr1Hz2acAb3OaL3l6DyKNg,RewAum_fQ0lXdFoUX18J0g,5,17,4,13,It is 4.5 stars. Not yet 5 stars. Mel has a go...,2020-08-18 19:09:46,Boon,...,27824,"2014,2015,2016,2017,2018,2019,20,20,2021","tuoDDqTDCB-lDgelvMrp5Q, 0yyXKmdI9lHxFKzomy6yfw...",387,4.04,Tampa,FL,33610,28.031316,-82.451534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39963,bZGNw6bx4cM5BwMe-CcT7A,gWZETLPu_ihAtLOYtrm0mw,Obk11u4CHhyqwkiJ517kaw,5,16,2,13,After asking friends for recommendations in Ne...,2022-01-10 16:10:13,Gabriella,...,1389,"20,20,2021","1auknNJG2nZYeZo9BFA4HQ, t5pBTw8QdHAwSLNLnh0Tmg...",167,4.33,New Orleans,LA,70112,29.953541,-90.074752
39971,cc8VettDuW-6nGLosYIIMA,qJMlmtF60eJnH2MWNps6Og,UM6XgOtTX4DWWcGDTWtjEg,5,0,0,0,We came to handlebar for geeks who drink trivi...,2019-11-15 17:52:32,Maggie,...,45,"2019,20,20,2021","zGe5pzwngTWnN1nI98Igsw, oRfbO4rQnPTp0auFprSCHA...",4,4.53,St Louis,MO,63110,38.627945,-90.251275
39986,s80hq-ipoqi9wgAQsfnR7g,6z_Kd-MrM66rzWQX9zZRjA,5gIfmupQTP3Lk4wIgNfoEQ,3,0,0,0,Had lunch there and the menu was very simple. ...,2016-08-10 21:37:23,Nancy,...,106,"2017,2018,2019,20,20,2021","LHNGyen_ZJ3sU1xQD9YwhQ, MR653vpwWLmDd9mFiTAU7w...",12,4.26,Fishers,IN,46037,39.928215,-86.024509
39986,s80hq-ipoqi9wgAQsfnR7g,6z_Kd-MrM66rzWQX9zZRjA,5gIfmupQTP3Lk4wIgNfoEQ,3,0,0,0,Had lunch there and the menu was very simple. ...,2016-08-10 21:37:23,Nancy,...,106,"2017,2018,2019,20,20,2021","LHNGyen_ZJ3sU1xQD9YwhQ, MR653vpwWLmDd9mFiTAU7w...",12,4.26,Fishers,IN,46037,39.928215,-86.024509


In [177]:
hometown = rJUJBloc.groupby(['user_id'])['city'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()

In [178]:
hometown

,city
user_id,
--_r6E98SNIrGU7weyNxbw,West Chester
--rpFxc_x14BWF708pfR6Q,Philadelphia
-0U1fhFq9zl4AyKafrKHFw,Nashville
-0aInSHjCWLfiNqfgmWnow,Philadelphia
-0aZWYi2YicFaLxTru96nA,Philadelphia
...,...
zwXmvn1op5LuFF2Kveqaug,Indianapolis
zxNtaKTMzZzum8ek2v4UXg,Reno
zxuxd6Hz2tKcpgZ71dYEcw,Edmonton


In [179]:
data = rJUJBloc.merge(hometown, how = 'left', on="user_id", suffixes=('_x', '_y'))

In [180]:
data['is_home'] = (data['city_x'] == data['city_y']).astype(int)

In [206]:
data['time_since'] = (pd.Timestamp(2022, 11, 29) - pd.to_datetime(data.date)).dt.days

In [209]:
elite = data.elite

In [216]:
elite.str.split(',')

0        [2012, 2013, 2014, 2015, 2016, 2017, 2018, 201...
1        [2014, 2015, 2016, 2017, 2018, 2019, 20, 20, 2...
2        [2014, 2015, 2016, 2017, 2018, 2019, 20, 20, 2...
3        [2014, 2015, 2016, 2017, 2018, 2019, 20, 20, 2...
4        [2014, 2015, 2016, 2017, 2018, 2019, 20, 20, 2...
                               ...                        
18863                                       [20, 20, 2021]
18864                                 [2019, 20, 20, 2021]
18865                     [2017, 2018, 2019, 20, 20, 2021]
18866                     [2017, 2018, 2019, 20, 20, 2021]
18867                                   [2016, 2017, 2018]
Name: elite, Length: 18868, dtype: object

In [219]:
data.elite

0        2012,2013,2014,2015,2016,2017,2018,2019,20,20,...
1                 2014,2015,2016,2017,2018,2019,20,20,2021
2                 2014,2015,2016,2017,2018,2019,20,20,2021
3                 2014,2015,2016,2017,2018,2019,20,20,2021
4                 2014,2015,2016,2017,2018,2019,20,20,2021
                               ...                        
18863                                           20,20,2021
18864                                      2019,20,20,2021
18865                            2017,2018,2019,20,20,2021
18866                            2017,2018,2019,20,20,2021
18867                                       2016,2017,2018
Name: elite, Length: 18868, dtype: object